In [ ]:
import itertools
from typing import Dict

from src.base.forecasting.models import TimeSeriesModel
from src.projects.fagradalsfjall.common.project_settings import (
    CV_HORIZON_SAMPLES,
    TS_CV_SPLITTER,
    TS_PRIMARY_METRIC,
)
from src.base.forecasting.models.time_series import TimeSeriesModelARMA
from src.projects.fagradalsfjall.common.dataset import load_train_data_numpy
from src.projects.fagradalsfjall.common.model_eval import ModelEvalResult
from src.projects.fagradalsfjall.common.model_repo import ModelRepo

# 1. Training - Cross-validation - Evaluation

In [ ]:
# -------------------------------------------------------------------------
#  Load TRAINING set
# -------------------------------------------------------------------------

print("Loading training dataset...    ", end="")
x_train = load_train_data_numpy()
print("Done.")

In [ ]:
# -------------------------------------------------------------------------
#  ARMA - Define & tune
# -------------------------------------------------------------------------

# --- ARMA settings -----------------------------------
final_run = True

if final_run:
    # -- FINAL --
    fit_robustness = 5
    pq_range = [0, 1, 2, 3, 4, 6, 8, 12, 16, 24, 32]
    wd_range = [0, 1e-2, 1e-1, 1.0]
else:
    # -- TEST --
    fit_robustness = 1
    pq_range = [0, 1, 2, 3]
    wd_range = [0]

# --- define model ------------------------------------
model_arma = TimeSeriesModelARMA(fit_robustness=fit_robustness)

arma_models = {
    "linear-arma": model_arma,
}  # type: Dict[str, TimeSeriesModel]

# --- run cross-validation ----------------------------
p_range = pq_range
q_range = pq_range
param_grid = {("p", "q"): [(p, q) for p, q in itertools.product(p_range, q_range) if p + q > 0], "wd": wd_range}

model_arma.cv.grid_search(
    x=x_train,
    param_grid=param_grid,
    metric=TS_PRIMARY_METRIC,
    ts_cv_splitter=TS_CV_SPLITTER,
    hor=CV_HORIZON_SAMPLES,
    retrain=True,
)

# --- model validation ------------------------------------
arma_model_eval_results = ModelEvalResult.eval_many(arma_models)

# --- save results ----------------------------------------
ModelRepo.save_models(arma_model_eval_results)

In [ ]:
# -------------------------------------------------------------------------
#  AR - Define & tune
# -------------------------------------------------------------------------

# --- AR settings -------------------------------------
final_run = True

if final_run:
    # -- FINAL --
    p_range = [1, 2, 3, 4, 6, 8, 12, 16, 24, 36, 48, 64, 96, 4 * 36, 4 * 48, 4 * 72]
    wd_range = [0.0, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0]
else:
    # -- TEST --
    p_range = [1, 2, 3, 4, 6, 8, 12, 16, 24, 36, 48, 64, 96]
    wd_range = [0.0, 1e-3, 1e-2, 1e-1, 1.0]

# --- define model ------------------------------------
model_ar = TimeSeriesModelARMA(q=0, fit_robustness=0)

ar_models = {
    "linear-ar": model_ar,
}  # type: Dict[str, TimeSeriesModel]

# --- run cross-validation ----------------------------
param_grid = {
    "p": p_range,
    "wd": wd_range,
}

model_ar.cv.grid_search(
    x=x_train,
    param_grid=param_grid,
    metric=TS_PRIMARY_METRIC,
    ts_cv_splitter=TS_CV_SPLITTER,
    hor=CV_HORIZON_SAMPLES,
    retrain=True,
)

# --- model validation ------------------------------------
ar_model_eval_results = ModelEvalResult.eval_many(ar_models)

# --- save results ----------------------------------------
ModelRepo.save_models(ar_model_eval_results)